# Matrix factorzation models for recommendation

The goal of this notebook is to get familiar with matrix factorization algorithms for recommender systems.

In [ ]:
import os
import sys
sys.path.append('../src/')

import pandas as pd
import numpy as np
from utils import *
import solutions

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

%load_ext autoreload
%autoreload 2

## Data exploration and preparation
Use case: we've taken the classic MovieLens data set, but we also gave it a twist to make it more applicable to the implicit rating use cases we encounter in the field. For instance, when we wish to recommend consumer products for a web store, we often do not have access to explicit user feedback (e.g. star ratings) and have to rely on historical purchase or click data only.
 
Therefore, we've binairzed the orgininal data set: all movie ratings with three or more stars are converted to rating 1.0 and all other watched and non-watched to 0.0 or NaN (implicit rating scenario). In particular:
* a 1.0 rating means that the user watched the trailer (impression) and subsequently also the whole movie (positive preference, implicit 1.0 rating)
* a 0.0 rating (optionally available through `drop_negatives=False`) means the user watched the trailer (impression), but decided not to watch the full movie (negative preference, implicit 0.0 rating), 
* NaN means the user didn't watch the trailer (no impression, unknown preference, implicit NaN or 0.0 rating). The original star rating can be thought of as a confidence measure for each rating, with higher ratings implying higher confidence that the user-item preference is positive.

In [ ]:
data = Dataset()
df = data.get_ratings(unary=True)
df.head()

In [ ]:
# here is some code to get item titles in case you wish to view them
item_desc = data.get_descriptions()
item_desc.head(3)

In [ ]:
# count number of ratings, unique users and items

### IMPLEMENT ###
n_ratings = NotImplemented
n_users = NotImplemented
n_items = NotImplemented

In [ ]:
print("{:,} users, {:,} items, {:,} ratings".format(n_users, n_items, n_ratings))

In [ ]:
# plot distribution of users-per-item
# plot distribution of items-per-user
# assumption: data doesn't contain duplicate ratings
# tip: use df.groupby()

_, (ax0, ax1) = plt.subplots(1, 2, figsize=(12, 4))

### IMPLEMENT ###
NotImplemented

As we can see, the number of users per item constitutes a long-tailed distribution, resembling the exponential distribution. We have to take note of the fact that there are a few very popular items (head) and a lot of niche items (tail). If we want to use our recommender to attend users to these niche items, we might want to give these items more weight, or filter the head in a postprocessing step. For now, we will continue using all items.

In [ ]:
# compute sparsity of user-item rating matrix
# ratio between number of rated user-item combinations and total user-item matrix size

### IMPLEMENT ###
sparsity = NotImplemented

In [ ]:
print("user-item rating matrix sparsity: {:.2f} %".format(sparsity * 100))

In [ ]:
from scipy.sparse import csr_matrix

# prepare data to a sparse matrix X
train_sel = df['set'] == 'train'
user_item = df.loc[train_sel, ['user','item']].values
users, items = user_item[:,0], user_item[:,1]
ratings = df.loc[train_sel, 'rating'].values

X = csr_matrix((ratings, (users, items)), shape=(data.n_users, data.n_items))
X

## Algorithms

Now it's time to implement a _factorization_ algorithm, where we decompose the user-item rating matrix into latent factors of users and items. One particularly useful flavor is low-rank non-negative matrix factorization, where the solution is constrained to comprise non-negative numbers, as is the case with our rating prediction problem. Here, the user-item rating matrix $X$ is factorized as a product of two lower rank matrices $W$ and $H$:
 
$$ X \approx W H $$

The low-rank property is very important, since it impplies that total information about the rating-property of the users is condensed in a much smaller volume of information. Thereby, the product of these lower-rank matrices will be less sparse that the original matrix, providing us with predictions for the unknown or zero entries. The rank is determined by the hyperparameter `n_components`.

Let's fit a non-negative matrix factorization model using the `NMF` class from `sklearn`.

In [ ]:
from sklearn.decomposition import NMF

factorizer = NMF(n_components=20, random_state=42)
W = factorizer.fit_transform(X)
H = factorizer.components_

In [ ]:
# predict top-3 item ratings for user 0
# compute dot product of the specific user (W) and all item factors (H)

### IMPLEMENT ###
predicted_ratings = NotImplemented

In [ ]:
# validate top 3
user0_top3 = np.argsort(predicted_ratings)[-3:][::-1]
print("top-3 items for user 0: {}".format([item_desc.loc[i] for i in user0_top3]))
np.testing.assert_equal(solutions.user0_top3, user0_top3)

The Frobenius norm of the difference between the reconstructed matrix and the original matrix $\sqrt{\sum\limits_{i}\sum\limits_{j}(w_i h_j - x_{ij})^2}$ may be viewed as the matrix reconstruction error after compression in the lower-rank components. Let's implement it below. Note that, for the sake of similicity, we can implement an inefficient implementation that instantiates the full dense matrix $X$. In real-life situations, you may want to avoid this situation and write a more memory-efficient implementation.

In [ ]:
def reconstruction_error(X, W, H):
    """frobenius norm of the matrix difference"""
    ### IMPLEMENT ###
    raise NotImplementedError

In [ ]:
e = reconstruction_error(X.toarray(), W, H)
print("reconstruction error: {:.2f}".format(e))
np.testing.assert_almost_equal(e, factorizer.reconstruction_err_)

We can now also plot the reconstruction error for different ranks of $W$ and $H$

In [ ]:
def error_at_rank(n_components):
    """compute reconstruction error as a function of n_components"""
    factorizer = NMF(n_components=n_components, random_state=42)
    W = factorizer.fit_transform(X)
    H = factorizer.components_
    return factorizer.reconstruction_err_

n_components = range(1, 42, 2)
pd.Series([error_at_rank(n) for n in n_components], index=list(n_components)).plot()
plt.xlabel('number of components (rank)')
plt.ylabel('reconstruction error')
plt.show()

As we can see, the steepest decline in reconstruction error is due to the first 10 components. Let's use this number from now on. Of course we can tune this hyperparameter using cross-validation to get optimal results. Feel free to implement this step below, after we have introduced percentile ranks.

## Evaluation

Now we are going to do some predictions on the test set and evaluate recommender quality.

An analysis of the distribution of the percentile ranks $rank_{ui}$ for all test items gives us an idea about the quality of recommendations. High percentile ranks (percentage of scores equal to or lower than $\hat{x}$, so high scores get high percentile ranks) are considered as most desirable for the user, while the lowest possible percentile rank 0% is considered as least preferred.

Since all items in the test set have been rated (purchased) by the user, our recommender is of good quality if most items in the test set have a percentile rank close to 100%. The average percentile rank of test items gives use a good indication of the overall quality.

In [ ]:
factorizer = NMF(n_components=10, random_state=42)
W = factorizer.fit_transform(X)
H = factorizer.components_

def percentile_ranks(x):
    """convert array to descending percentile ranks"""
    temp = x.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(x))
    ranks = ranks / (len(x) - 1) * 100
    return ranks

def item_percentile_rank(user_item, W, H):
    """for a single test item, compute the percentile rank using all item predictions"""
    user, item = user_item[0], user_item[1]
    x_pred = W[user,:].dot(H)
    return percentile_ranks(x_pred)[item] 

user_item_test = df.loc[~train_sel, ['user','item']].values
ratings_test = df.loc[~train_sel, 'rating'].values

ranks = np.apply_along_axis(item_percentile_rank, axis=1, arr=user_item_test, W=W, H=H)
print("average percentile rank: {:.2f}%".format(ranks.mean()))

We can also apply signal detection theory to these percentile ranks, and obtain a receiver-operator-characteristic (ROC) curve for our recommender.

In [ ]:
edges = np.arange(0, 101, 1)
def ranks_to_cdf(ranks):
    """convert percentile ranks to a cumulative distribution function"""
    hist, _ = np.histogram(100 - ranks, normed=True, bins=100)
    return np.insert(np.cumsum(hist), 0, 0)

# results data frame
results = pd.DataFrame(index=edges)

# store results from a random recommender
results['Random'] = ranks_to_cdf(np.random.rand(1000,) * 100)

# store results of the NMF recommender
results['NMF'] = ranks_to_cdf(ranks)

def plot_cdf(df):
    """plot the columns of a data frame with percentiles as index"""
    df.plot()
    plt.title('Cumulative distribution function \n of test item recommendation probability')
    plt.xlabel('top % recommended')
    plt.ylabel('probability (recall)')
    plt.show()
    
plot_cdf(results)    

In [ ]:
from sklearn.metrics import auc
# compute the area-under-the-curve (AUC) scores for this model, to obtain a single metric that may be optimized
# tip: use the results data frame from above
# divide by 100 to map the percentages to proportions

def area_under_the_curve(series):
    """area under the curve of cumulative distribution function of test item recommendation probability"""
    ### IMPLEMENT ###
    raise NotImplementedError

In [ ]:
auc_score = area_under_the_curve(results['NMF'])
print("AUC score: {:.2f}".format(auc_score))
np.testing.assert_almost_equal(solutions.auc_score, auc_score, decimal=1)

Actually, the most interesting part of this graph is located in the left lower corner. We usually don't have the oppertunity to recommend thousands of items to a user, but rather only a couple. Therefore, other metrics weight the items at the top of the list more than items at the bottom of the list. One of these metrics is the [Mean reciprocal rank](https://en.wikipedia.org/wiki/Mean_reciprocal_rank):
$$
MRR = \frac{1}{|Q|}\sum\limits_{q=1}^{Q}{\frac{1}{rank_q}}
$$

In [ ]:
# compute the mean reciprocal rank of this model
# note that we first have to convert back the descending percentile ranks to ascending absolute ranks

def mrr(x):
    """mean reciprocal rank"""
    x =  n_items - ((x / 100) * (n_items - 1))
    
    ### IMPLEMENT ###
    raise NotImplementedError

In [ ]:
mrr_score = mrr(ranks)
print("mean reciprocal rank: {:.2f}".format(mrr_score))
np.testing.assert_almost_equal(solutions.mrr_score, mrr_score, decimal=2)

## Comparing models

First, we need a better baseline for our model. A very important baseline model is the popular item recommender. This  algorithm simply recommends the most popular items to all users. Usually in recommendation experiments, and in particular when dealing with implicit feedback data, this non-personalized model already has a fair amount of predictive power.

Let's create some helper classes to wrap our factorizers and other recommender models.

In [ ]:
class PopularItemsRecommender(Recommender):
    
    def fit(self, user_item, ratings):
        users, items = user_item[:,0], user_item[:,1]
        X = csr_matrix((ratings, (users, items)), shape=(self.n_users, self.n_items))
        self.popularity = np.asarray(X.mean(axis=0)).flatten()
        self.ranks = percentile_ranks(self.popularity)
        return self
        
    def predict(self, user_item):
        items = user_item[:,1]
        return np.array([self.popularity[i] for i in items])

    def percentile_rank(self, user_item):
        items = user_item[:,1]
        return np.array([self.ranks[i] for i in items])

In [ ]:
class FactorizationRecommender(Recommender):
    
    def __init__(self, dims, factorizer):
        super().__init__(dims)
        self.factorizer = factorizer
        
    def fit(self, user_item, ratings, **kwargs):
        users, items = user_item[:,0], user_item[:,1]
        X = csr_matrix((ratings, (users, items)), shape=(self.n_users, self.n_items))
        self.W = W = self.factorizer.fit_transform(X, **kwargs)
        self.H = H = self.factorizer.components_
        self.reconstruction_err_ = reconstruction_error(X.toarray(), W, H)
        return self

    def predict(self, user_item):
        users, items = user_item[:,0], user_item[:,1]
        ratings = np.zeros(len(users,))
        for n, (user, item) in enumerate(zip(users, items)):
            ratings[n] = self.W[user, :].dot(self.H[:, item])
        return np.array(ratings)

    def percentile_rank(self, user_item):
        # note that this method uses the item_percentile_rank function defined in this notebook earlier
        return np.apply_along_axis(item_percentile_rank, axis=1, arr=user_item, W=self.W, H=self.H)

In [ ]:
def fit_and_evaluate(model, results, name, **kwargs):
    """to fit a model and store the performance results"""
    
    print("\n", name)
    
    model.fit(user_item, ratings, **kwargs)
    model.evaluate(user_item_test, ratings_test)
    
    ranks = model.percentile_rank(user_item_test)
    results[name] = ranks_to_cdf(ranks)
    
    # optionally print additional metrics here
    print("AUC score: {:.2f}".format(area_under_the_curve(results[name])))
    print("mean reciprocal rank: {:.2f}".format(mrr(ranks)))
    
    return results

In [ ]:
from sklearn.decomposition import TruncatedSVD

model = PopularItemsRecommender(dims=(data.n_users, data.n_items))
results = fit_and_evaluate(model, results, name='PopularItems')

factorizer = TruncatedSVD(n_components=30, random_state=42)
model = FactorizationRecommender(dims=(data.n_users, data.n_items), factorizer=factorizer)
results = fit_and_evaluate(model, results, name='TruncatedSVD')

In [ ]:
# experiment with different factorizers and/or hyperparamter settings

## IMPLEMENT ###
# factorizer =
# model = 
# results = fit_and_evaluate(model, results, name='MyModel')

In [ ]:
plot_cdf(results)

## Item similarities

In [ ]:
model = FactorizationRecommender(dims=(data.n_users, data.n_items), factorizer=NMF(n_components=10, random_state=42))
model.fit(user_item, ratings)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

### IMPLEMENT ###
item_similarities = NotImplemented

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(item_similarities)
plt.title('item-item similarity matrix')
cb = plt.colorbar(fraction=0.03, pad=0.05)
cb.ax.set_title('cosine \n similarity \n')
plt.show()

Observe the clusters of similar movies, according to their latent factors. 

We can now inspect the top-n most similar items for a given item.

In [ ]:
n = 5
item = 10

item_top = np.argsort(item_similarities[item,:])[-n:][::-1]
print("top-{} most similar items for {}: \n {}".format(n, item_desc.loc[item], [item_desc.loc[i] for i in item_top]))

It is a known issue that the latent factors are heavily biased by popularity. The quality of item-item similarities may be improved by removing the popularity effect from the user-item rating matrix, for instance using spectral clustering techniques. Google for more info and feel free to implement here to compare.

## Matrix factorization mechanics

Now it's time to dive into the math of the matrix factorization algorithms.

We find our decomposed matrix by minimizing the following cost function: 

$$ \mathcal{L} = \frac{1}{2} \times ||X - WH||_F^2+ \frac{1}{2} \times\lambda \times||W||_F^2+ \frac{1}{2} \times\lambda \times||H||_F^2$$

The following computation yields the gradients:

$$ \nabla_{W}\mathcal{L} = WHH^T-XH^T +  \lambda  \times W $$
$$ \nabla_{H}\mathcal{L} = W^TWH - W^TX +  \lambda  \times H  $$

We could of course implement gradient descent to solve (why not try later as exercise?), but we could also guess value of either W or H and then solve the quadratic problem for the other. So either we fix $H$ and solve $ \nabla_{W}\mathcal{L} = 0 $ or fix $W$ and solve $ \nabla_{H}\mathcal{L} = 0 $

Both these equations are linear in the unknowns. At each step n, we apply the following transformations:

$$ W_{(n+1)}=XH_{(n)}^T(H_{(n)}H^T_{(n)}+\lambda \mathbb{1})^{-1} $$
$$ H_{(n+1)}=( W^T_{(n)}W_{(n)}+\lambda \mathbb{1})^{-1} W^T_{(n)}X $$

In [ ]:
from sklearn.base import TransformerMixin

class AlternatingLeastSquaresFactorizer(BaseEstimator, TransformerMixin):
          
    def __init__(self, n_components, n_iterations=10, l2=0.001):
        
        self.n_iterations = n_iterations
        self.n_components = n_components
        self.l2 = l2
          
    def fit_transform(self, X, sample_weights=None):
        X = X.toarray()
        l2 = self.l2
        n_components = self.n_components
        n, m = X.shape
        
        # initate W and H as random matrices
        W = np.random.rand(n, n_components)
        H = np.random.rand(n_components, m)
        
        # store sample weights for later use
        C = sample_weights
        
        # computation
        for it in range(self.n_iterations):
            
            if sample_weights is None:
                
                W = np.dot(X, H.T).dot(np.linalg.pinv(np.dot(H, H.T) + l2 * np.eye(n_components)))
                H = np.linalg.pinv(np.dot(W.T, W) + l2 * np.eye(n_components)).dot(np.dot(W.T, X))
                
            else:
                ### IMPLEMENT ###
                raise NotImplementedError('Weighted ALS not implemented yet.')

            if it % 100 == 0:
                print("iteration: {}\t error: {:.4f}".format(it, reconstruction_error(X, W, H)))
        
        self.components_ = H
        return W

In [ ]:
# a small test with random data
model = AlternatingLeastSquaresFactorizer(n_iterations=201, n_components=20, l2=0.001)
_ = model.fit_transform(csr_matrix(np.random.rand(200, 100)))

The above custom `AlternatingLeastSquaresFactorizer` class works, but we've merely created something similar to (but more inefficient than) the scikit models. Let's customize our loss function to incoorpoate confidence information about the ratings (see [Hu et al. 2011](http://yifanhu.net/PUB/cf.pdf) for more information).

Specifically, we can weigh the error with a confidence measure. A way to do this is by defining this confidence as a function of some user preference indicator, such as the number of purchases or views.  We can use the initial star ratings for this purpose. Let $c_{ij}$ be:

$$c_{ij} = 1+ \alpha r_{ij} $$

Where $r_{ij}$ is the initial star rating of the user. The error is then weighted as follows:

$$ ||X||_C = \sum_{ij}C_{i,j} x_{i,j}^2 $$

A quick calculation shows that the alternating least-square algorithm has the form,

$$ W_{ij}^{(n+1)}=e_i^TXC^{(i)}H_{(n)}^T(H_{(n)}C^{(i)}H^T_{(n)}+\lambda \mathbb{1})^{-1}e_j $$
$$ H_{ij}^{(n+1)}=e_i^T( W^T_{(n)}C^{(i)}W_{(n)}+\lambda \mathbb{1})^{-1} W^T_{(n)}C^{(i)}X e_j$$

Try to implement the confidence weighted ALS algorithm in the `AlternatingLeastSquaresFactorizer` class above. We've already added a routine where the solver can be implemented. Good luck.

In [ ]:
# prepare confidence measures to match ratings
alpha = 40
c = df.loc[train_sel, 'confidence'].values * alpha
c = csr_matrix((c, (users, items)), shape=(data.n_users, data.n_items)).toarray()
confidences = np.ones((data.n_users, data.n_items)) + c

# define model
params = dict(n_components=10, n_iterations=3, l2=0.001)
model = FactorizationRecommender(dims=(data.n_users, data.n_items),
                                 factorizer=AlternatingLeastSquaresFactorizer(**params))

results = fit_and_evaluate(model, results, name='WeightedALS', sample_weights=confidences)

In [ ]:
plot_cdf(results)

Feel free to play around with the models, customize them, tune them, and try to beat the vanilla `NMF` with 10 components.

## Big data libraries

Obviously, these models thrive with more data, and we've only explored the smallest release of the MovieLens dataset. Do note that the computation of the matrix factorization models quickly increases with more data. Therefore, it is advised to implement these models using scalable machine learning frameworks such as Apache Spark or TensorFlow. Here are a couple of links to get you started with matrix factorization models using these frameworks.

### Spark
* https://spark.apache.org/docs/2.2.0/mllib-collaborative-filtering.html
* http://ampcamp.berkeley.edu/big-data-mini-course/movie-recommendation-with-mllib.html

### TensorFlow
* http://willwolf.io/2017/04/07/approximating-implicit-matrix-factorization-with-shallow-neural-networks/  
  Note that [Network architecture #1](http://willwolf.io/2017/04/07/approximating-implicit-matrix-factorization-with-shallow-neural-networks/#Network-#1) shows our matrix factorization algorithm in neural network form.
* http://katbailey.github.io/post/matrix-factorization-with-tensorflow/

Feel free to try either one of these libraries and compare with this notebook.

That's it.

We hope you liked this notebook!

Cheers,

The hands-on Data Scientists from [BigData Republic](https://www.bigdatarepublic.nl)